In [23]:
from thefuzz import fuzz, process
import pandas as pd

#import function to query mysql server
import Functions.datamart_query as q

#pull in account data from SQL file based on region
with open(f'/Users/jnichols/Python Files/Fuzzy Match/Queries/Ireland.sql', 'r') as myfile:
    query=myfile.read().replace('\n', ' ')
# Store query results in accounts df.
db = q.run_query(query, database='Production')

m = pd.read_excel(f'/Users/jnichols/Python Files/Fuzzy Match/IrelandMissingAcctsV2.xlsx') 



/Users/jnichols/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [24]:
matches = pd.DataFrame(columns = ['MatchName', 'MatchScore'])
for i in range(len(m)):
    acc = m.at[i,'AccountName']
    match = db['AccountName']
    tup = process.extractOne(acc, match, scorer = fuzz.token_sort_ratio)
    a, b, c = tup
    matches.at[i,'MatchName'] = a
    matches.at[i,'MatchScore'] = b

print(matches)


                                           MatchName MatchScore
0                                   NOA NEWS LIMITED         55
1                  Three Ireland (Hutchison) Limited        100
2                                   Allcare Pharmacy         69
3                                            Empower         40
4                                     Dublin Unified         64
..                                               ...        ...
236  Galway / Roscommon Education and Training Board         59
237                Waterford Institute of Technology        100
238                                WATERWAYS IRELAND        100
239               NORTHERN IRELAND SCREEN COMMISSION         66
240                             BAXTER HEALTHCARE SA         53

[241 rows x 2 columns]


In [25]:
accounts_with_matches = pd.merge(m, matches, how='left', left_index=True, right_index=True)

In [27]:
accounts_with_matches = pd.merge(accounts_with_matches, db, how = 'left', left_on = 'MatchName', right_on = 'AccountName')

In [28]:
print(accounts_with_matches)

                                         AccountName_x  \
0                                        Bidest Noonan   
1                    Three Ireland (Hutchison) Limited   
2                                     Lloylds Pharmacy   
3                                                 PWC    
4                                          United Drug   
..                                                 ...   
236  Waterford and Wexford Education and Training B...   
237                  Waterford Institute of Technology   
238                                  Waterways Ireland   
239                     Workplace Relations Commission   
240                               Údarás na Gaeltachta   

                                           MatchName MatchScore  \
0                                   NOA NEWS LIMITED         55   
1                  Three Ireland (Hutchison) Limited        100   
2                                   Allcare Pharmacy         69   
3                                  

In [29]:
accounts_with_matches.to_csv('AccountsWithMatches.csv')